In [40]:
import xarray as xr
import hvplot.xarray
from dask.distributed import Client
import numpy as np
import pandas as pd
import datetime

In [41]:
infile = r'C:\Users\xavier.mouy\Documents\PhD\Projects\Dectector\DFO_RCA_run\hourly_detections_april-july-2019.nc'

In [42]:
ds = xr.open_dataset(infile)
fs_hourly_count = xr.DataArray(ds.label_class, coords={'date': ds.date}, dims='date')
fs_hourly_count

<xarray.DataArray 'label_class' (date: 1815)>
array([ 238,  309,  187, ..., 2119, 1841,  399])
Coordinates:
  * date     (date) datetime64[ns] 2019-04-10T15:00:00 ... 2019-06-25T05:00:00

In [43]:
fs_hourly_count.hvplot(width=800)

:Curve   [date]   (label_class)

In [44]:
# Date vector
date = list(fs_hourly_count.resample(date='D').groups.keys())

# hour of day vector
#hour = pd.date_range("00:00", freq="1H", periods=24).time
hour = range(0,24,1)

# detections matrix
detections = np.empty((len(hour),len(date),))
detections[:] = np.nan
detec_matrix = xr.DataArray(detections, coords={'hour': hour, 'date': date}, dims=['hour','date'])

# Fill in 2D Array
for x in fs_hourly_count:
    fulldate = pd.to_datetime(x.date.data)
    day_str = datetime.datetime.strftime(fulldate,'%Y-%m-%d')
    hour = fulldate.time().hour
    nDetec = x.data
    detec_matrix.loc[hour,day_str] = nDetec

# Sum of detec per day
detec_sum = detec_matrix.sum(dim='hour')

In [45]:
# plot
detec_matrix.hvplot(width=800)

:Image   [date,hour]   (value)

In [49]:
fs_hourly_count.resample(date='D').sum().hvplot(kind='step', width=820)

:Curve   [date]   (label_class)